<a href="https://colab.research.google.com/github/sukantamitra007/GenAILearning/blob/TransformerModel/Project_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lets Install Necessary libraries

In [1]:
pip install transformers[torch]  datasets evaluate rouge_score

# Load BillSum dataset
I am loading some sample data to use fine tuning concent on my data set. instead of preparing new data i am using exiting data to train summarization model

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.99)
billsum_train= billsum['train']   # Doing this to reduce the data volumn
billsum = billsum_train.train_test_split(test_size=0.40)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Preprocess

In [3]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# preprocessing function

In [4]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]  # Add prefix  as required for the model
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) # add truncation on and max length

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True) # add target truncation on

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
tokenized_billsum = billsum.map(preprocess_function, batched=True) #setting batched=True to process multiple elements of the dataset at

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint) #dynamically pad the sentences to the longest length in a batch during collation.

# Evaluate

In [7]:
import evaluate
rouge = evaluate.load("rouge")

In [8]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train

In [9]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_project",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False#,
    #push_to_hub=True,
)

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("SummarizationFineTunedModel")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,5.119209,0.144200,0.035500,0.125300,0.125300,19.000000
2,No log,5.070542,0.144200,0.035500,0.125300,0.125300,19.000000
3,No log,5.036646,0.144200,0.035500,0.125300,0.125300,19.000000
4,No log,5.019797,0.144200,0.035500,0.125300,0.125300,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [13]:
from transformers import pipeline
summarizer = pipeline("summarization", model="SummarizationFineTunedModel")
#summarizer(text,max_length=51)



Use Text Classification model funed tuned with financial news. In realtime we will finetuned it with production orginal customer review and the predefined label for our company. But for this project I am using existing model . As I have already fine tuned one model to showcase how to do fine tuning.

In [36]:
from transformers import pipeline

pipe1 = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [37]:
def clssification_Of_review(feedback):
  classified_text=pipe1(feedback)[0].get('label')
  return classified_text

In [17]:
from transformers import pipeline, AutoTokenizer
import torch

torch.manual_seed(0)
model = "openai-community/gpt2" #"tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [18]:
def product_suggestion(review_summary):
  prompt = f"""You are a product engineering and based on the input text provide a improvement idea to the company.
  Text: The screen is prone to scratches and cracks. I've only had this phone for a week, and it already looks beaten up.
  Improvement idea: Product team need to work on screen quality improvement.
  Text: The build quality is poor. The phone feels flimsy and cheaply made, and it's prone to overheating.
  Improvement idea: Product team need to look into hardware improvement.
  Text: {review_summary}
  Final Output:"""

  sequences = pipe(
      prompt,
      max_new_tokens=40,
      do_sample=True,
      top_k=10,
  )
  return sequences[0].get('generated_text').split("Final Output:")[1]


**User Interface for summarization and sentiment object**

In [ ]:
pip install gradio

In [47]:
import gradio as gr
def summarizer_new(inputFeedback):
    summary_text=summarizer(inputFeedback,max_length=51)[0].get('summary_text') #Summarization Model
    classification_label=clssification_Of_review(summary_text) #Classification Model
    if classification_label in 'negative':
      product_suggestion_text=product_suggestion(summary_text) #Product Suggestion
    else:
      product_suggestion_text= "Customer is happy with the product."

    return summary_text,classification_label,product_suggestion_text

demo = gr.Interface(fn=summarizer_new, inputs=gr.Textbox(label="Feedback", lines=3), outputs= [gr.Textbox(label="summary", lines=3)
                                                                      , gr.Textbox(label="sentiment", lines=3)
                                                                      ,gr.Textbox(label="product suggestion", lines=3)
                                                                      ])
demo.launch()


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b53c4ea254ab822b65.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
